## Exercices de découverte du PMSI MCO (Version Python)
Bienvenue sur ce notebook d'introduction à la manipulation du PMSI ! L'objectif est de vous accompagner, grâce à une manipulation guidée, dans la découverte de quelques tables PMSI et plus spécifiquement celles du champ MCO (Médecine-Obstétrique-Chirurgie). Les autres champs du PMSI - auxquels nous nous intéresserons pas dans les exercices suivants - sont les champs HAD (hospitalisation à domicile), SSR (soins de suite et réadaptation) et RIM-P (psychiatrie). Vous trouverez le schéma des concepts de fonctionnement du PMSI et l'articulation de ses principales bases sur la fiche dédiée.Ces quelques exercices sont conçus pour vous donner une première prise en main des données disponibles et de leur structure, en attendant que vous ayez accès aux données réelles dont vous avez besoin pour vos projets. Les exercices ici privilégient une approche par établissement de soins et non par patient (bien qu'elle soit aussi possible !). A noter que les données utilisées pour le développement de ce notebook sont des données synthétiques avec les mêmes formats et types des valeurs des données réelles du SNDS. Il s'agit de données fictives générées aléatoirement, qui ne sont pas nécessairement plausibles et ne recouvrent pas une réalité médicale. Ces données ont été générées par le Lab Santé de la DREES.Pour plus d'information sur ces données, vous pouvez consulter la page dédiée sur le site de documentation du SNDS. Préparation et configuration
Commençons par importer le module pandas pour exécuter le code.

In [1]:
 # Chargement des modules

import pandas as pd   

 Importons maintenant les données synthétiques sur lesquelles nous allons exécuter les exercices./!\ Attention, les données synthétiques ne limitent pas les données d'une table à une période temporelle donnée. Cela est toutefois le cas dans le SNDS où la plupart des tables du PMSI sont annuelles.

In [3]:
from google.colab import drive
drive.mount('/drive')


Mounted at /drive


In [8]:
# Récupération des fichiers de données depuis Gitlab
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/snds_&_covid/snds/'

In [11]:
# Chemins d'accès aux fichiers
df_mco_b = pd.read_csv(data_dir + "T_MCOaa_nnB.csv", header=0, sep=",")         # Description du séjour
df_mco_a = pd.read_csv(data_dir + "T_MCOaa_nnA.csv", header=0, sep=",")         # Actes CCAM
df_mco_c = pd.read_csv(data_dir + "T_MCOaa_nnC.csv", header=0, sep=",")         # 
df_mco_d = pd.read_csv(data_dir + "T_MCOaa_nnD.csv", header=0, sep=",")         # 
df_mco_e = pd.read_csv(data_dir + "T_MCOaa_nnE.csv", header=0, sep=",")         # 
df_mco_fb = pd.read_csv(data_dir + "T_MCOaa_nnFB.csv", header=0, sep=",")       # Prestations Objectifs Quantifiés Nationaux
df_mco_fbstc = pd.read_csv(data_dir + "T_MCOaa_nnFBSTC.csv", header=0, sep=",") # Prestations Actes et Consultations Externes

# Création de la variable sejour unique (couple ETA_NUM, RSA_NUM)
df_mco_b["ETA_NUM_RSA_NUM"] = df_mco_b["ETA_NUM"].astype(str) + "_" + df_mco_b["RSA_NUM"].astype(str)
df_mco_a["ETA_NUM_RSA_NUM"] = df_mco_a["ETA_NUM"].astype(str) + "_" +  df_mco_a["RSA_NUM"].astype(str)
df_mco_c["ETA_NUM_RSA_NUM"] = df_mco_c["ETA_NUM"].astype(str) + "_" + df_mco_c["RSA_NUM"].astype(str)
df_mco_d["ETA_NUM_RSA_NUM"] = df_mco_d["ETA_NUM"].astype(str) + "_" +  df_mco_d["RSA_NUM"].astype(str)

In [15]:
df_mco_b.head()

,Unnamed: 0,AGE_ANN,AGE_GES,AGE_JOU,ANT_SUP_NBR,AUT_PGV_NBR,BDI_COD,BDI_DEP,BEB_SEJ,BEH_NBJ,CAI_SUP_NBR,COD_SEX,DEL_REG_ENT,DGN_PAL,DGN_REL,DOS_TYP,ENT_MOD,ENT_PRV,ETA_NUM,ETE_GHS_NUM,EXB_NBJ,GHS_9615_ACT,GHS_HS_INNOV,GHS_NUM,GRC_GHM,GRC_RET,GRC_VER,GRG_GHM,GRG_RET,GRG_VER,INNOV_NUM,MACH_TYP_RAD,NBR_ACT,NBR_DGN,NBR_RUM,NBR_SEA,NBR_SUP_NN1,NBR_SUP_NN2,NBR_SUP_NN3,NBR_SUP_REA,...,SOR_MOD,SOR_MOI,SUP_ENT_DPA,SUP_ENT_DPC,SUP_ENT_HEM,SUP_HEM_HS,SUP_RAD_PED,TAR_SEQ_NUM,TOP_AVASTIN,TOP_DEF_CARD,TOP_GHS_MIN_SUS,TOP_VLV_AOR,TYP_GEN_RSA,UHCD_TOP,NBR_NAIS_ANT,GHS_9512_ACT,GHS_9515_ACT,GHS_9511_ACT,GHS_9619_ACT,GHS_9610_ACT,NBR_IVG_ANT,GHS_9620_ACT,NBR_SUP_SSC,GHS_9622_ACT,GHM_24707Z_ACT,GHS_9621_ACT,NBR_SEA_SROS,GHM_24706Z_ACT,GHM_24705Z_ACT,FAIS_NBR,GHS_9611_ACT,GHS_9612_ACT,GHS_9510_ACT,GHS_9524_ACT,DLY_ACT,GHS_6523_ACT,NBR_SUP_SRA,COD_IGS,ANN_IVG_PREC,ETA_NUM_RSA_NUM
0,0,118,26,111,461,1,01290,75,NaN,3875,49,2,611,F4300,F3231,3,-1,7,713198774,6164,119,46,9823,5917,SGLMhV,1,13,jqaGZN,1,11,HChkthwZDruZCln,1,1415,87,69,87,175,67,131,82,...,9,5,8,31,0,7,5,3,0,1,1,0,0,1,43,35371173,6093587,36903698,724277,59747822,0,17582530,93224615,75509058,495,38522366,52892110,957,870,37421922,98299090,34495583,71426457,45566028,Adj,56945313,44084600,OP,1920,713198774_7743808116
1,1,29,30,325,666,0,03450,30,L,6688,44,7,530,F2050,F3230,1,6,4,167591733,1139,166,63,9243,2738,kxPJXJ,1,17,xHGDSb,1,11,nQBakJdKQmqVTvr,1,2432,95,86,61,188,193,131,535,...,-2,6,24,11,14,73,1,4,1,0,1,1,3,0,90,48978082,45402706,44310944,61783483,38708336,47,49957282,93628160,42832341,922,20999550,44143260,157,623,3359836,73096275,44728551,39757043,43191223,Xh,81591176,11792498,shT,1919,167591733_9286683630
2,2,12,38,271,295,0,05C04,03,NaN,7193,13,8,930,F2019,F2052,2,-2,5,370368029,3386,54,40,2741,8621,mMXlfY,1,-1,PjLyum,0,11,oEFCygNdQtbimjl,4,2330,84,23,74,211,56,349,807,...,8,4,3,17,15,19,25,5,0,0,0,0,1,0,65,11302474,84326395,65603055,21291900,77666384,27,71071926,55657630,99933448,394,95438146,91840647,226,202,12254531,56456677,78425128,45863994,55958633,C,20018232,29630164,NaN,1927,370368029_2968714597
3,3,14,23,23,968,2,04100,32,NaN,3982,23,1,16,F3131,F1953,2,7,3,863007477,1288,307,85,6199,1859,ObSJLR,1,19,jMpVjZ,0,11,onTMhuUfEAXVNOi,2,2090,84,40,67,10,174,31,256,...,-1,2,7,6,2,91,27,4,1,0,0,0,3,1,69,85606252,25460223,31961990,95646710,34171217,93,9756455,12992522,96462431,950,34795484,42148802,855,769,97885085,17680722,49586424,3485009,85979033,mZ,26756097,33122260,x,1919,863007477_6525685393
4,4,24,25,225,893,2,01990,95,E,3840,43,2,175,F429,F2053,4,6,1,291447785,4328,85,71,6978,2571,sWSMyE,0,20,bgezwn,0,11,lRPcSmFKZiXASGQ,1,1409,37,54,20,3,75,110,256,...,-2,1,23,23,11,57,14,3,0,1,0,1,1,0,44,11500797,84452732,61421275,96179249,52448855,9,6266916,36045359,61042164,332,14537826,89082688,255,643,82193819,27524215,3515126,52995984,69991466,Q,67886774,37902007,xw,1931,291447785_2694770682


 Dans le SNDS, des nettoyages sont nécessaires pour la bonne exploitation des données. Au niveau du champ MCO du PMSI, l'un d'entre eux consiste à supprimer les doublons au niveau de certains établissements de soins grâce à leur numéro FINESS. En effet, pour l’APHP (Assistance Publique - Hôpitaux de Paris), les HCL (Hospices Civils de Lyon) et l’APHM (Hôpitaux Universitaires de Marseille), les établissements sont présents à la fois sous leur code FINESS géographique et juridique (jusqu’en 2018). Afin de ne pas les compter en double, il faut supprimer un certain nombre de codes grâce à la variable ETA_NUM renseignant le code FINESS des établissements. Vous trouverez ci-dessous un vecteur eta_num_supr avec les codes ETA_NUM à supprimer lorsqu'on travaille sur le PMSI (cf. Fiche Dépenses des établissements de santé publics dans le PMSI).

In [14]:
ETA_NUM_doubles = [
'130780521','130783236', '130783293', '130784234', '130804297', '600100101','750041543', '750100018',
'750100042', '750100075', '750100083', '750100091', '750100109', '750100125','750100166', '750100208', 
'750100216', '750100232', '750100273', '750100299', '750801441', '750803447','750803454', '910100015', 
'910100023', '920100013', '920100021', '920100039', '920100047', '920100054','920100062', '930100011', 
'930100037', '930100045', '940100027', '940100035', '940100043', '940100050','940100068', '950100016', 
'690783154', '690784137', '690784152', '690784178', '690787478', '830100558' 
]

## Exercice 1 : Nombre de séjours hospitaliers par établissement
Dans cet exercice, nous allons calculer le nombre de séjours hospitaliers par établissement l'année aa. L'exercice fait donc appel à la table des séjours T_MCOaaB. Un séjour hospitalier est identifié par le couple de variables ETA_NUM (numéro FINESS de l'établissement) - RSA_NUM (numéro séquentiel du résumé de sortie anonyme). Comme évoqué, à coté de la suppression des séjours en double pour l'APHP, l'AP-HM et les HCL (filtre ~ df_mco_b.index.get_level_values(0).isin(ETA_NUM_doubles)), il faut aussi penser à supprimer les séjours en erreur (filtre df_mco_b["GRG_GHM"].str[0:2] != "90").

In [17]:
filtres_mco_b = (
    (df_mco_b["GRG_GHM"].str[0:2] != "90")         # Suppression des séjours en erreur
    &
    (~
     df_mco_b.index.get_level_values(0)            # Suppression des séjours en double APHP, HCL, AP-HM
     .isin(ETA_NUM_doubles))                      
)

sej_par_etab = df_mco_b[filtres_mco_b].groupby('ETA_NUM')['RSA_NUM'].nunique() # Calcul du nombre de séjours par établissement
sej_par_etab = sej_par_etab.reset_index(name='NB_SEJ')
sej_par_etab.head(5)                                                          # 20 premières lignes de la table de sortie

,ETA_NUM,NB_SEJ
0,101562418,19
1,103942560,18
2,108502982,15
3,110914559,16
4,111826119,30


## Exercice 2 : Nombre de séjours hospitaliers suite à un passage aux urgences par établissement
Dans cet exercice, nous allons calculer le nombre de séjours hospitaliers par établissement qui font suite à un passage aux urgences l'année aa. L'exercice fait donc appel à la table des séjours T_MCOaaB.Nous allons reprendre le bout de code précédent avec une sélection supplémentaire sur les séjours dont le mode d'entrée (ENT_MOD) et la provenance (ENT_PRV) sont les urgences.

In [18]:
pass_urg_hospi = df_mco_b[
                          filtres_mco_b  # cf exercice 1            # Suppression des séjours en erreur et des séjours en double APHP, HCL, AP-HM
                          & (df_mco_b["ENT_MOD"]== 8)               # Mode entrée 8 (urgences)
                          & (df_mco_b["ENT_PRV"]== 5)               # Provenance 5 (urgences)
                          ].groupby('ETA_NUM')['RSA_NUM'].nunique() # Calcul du nombre de séjours après passage aux urgences par établissement
pass_urg_hospi = pass_urg_hospi.reset_index(name='NB_URG_HOSP')
pass_urg_hospi.head(5)  

,ETA_NUM,NB_URG_HOSP
0,108502982,2
1,111826119,2
2,129827540,2
3,156421106,1
4,161884379,1


## Exercice 3 : Repérer les passages aux urgences (avec et sans hospitalisation)
Dans cet exercice, nous allons calculer le nombre de passages aux urgences par établissement l'année aa.
Les passages aux urgences avec hospitalisation (en établissement public ou privé) se trouvent dans la base T_MCOaaB. Nous avons déjà réalisé leur repérage dans l'exercice 2.
Pour les passages aux urgences sans hospitalisation, il faudra récupérer l'information dans les actes et consultations externes :
T_MCOaaFBSTC pour les établissements publicsT_MCOaaFB pour les établissements privés
Notons que pour les tables T_MCOaaFBSTC et T_MCOaaFB, le repérage des passages aux urgences se fait grâce à la ligne de code ACT_COD=='ATU' qui correspond au code activité "forfait d'accueil et de traitement des urgences". Il faut ensuite sommer les activités pour en déterminer le nombre en utilisant la variable quantité d'acte ACT_NBR. Par ailleurs, il n'est pas nécéssaire d'appliquer les filtres sur les séjours car il n'y a pas eu de séjour hospitalier. De plus, dans la table T_MCOaaFB uniquement, il n'est pas nécéssaire d'appliquer le filtre pour la suppression des doublons APHP, HCL, AP-HM car il s'agit des établissements privés.

In [19]:
# Passages urgences avec hospitalisation (idem exercice 2)
pass_urg_hospi

# Passages aux urgences sans hospitalisation (sans séjour) dans le public
filtres_mco_fbstc = (
    (df_mco_fbstc["ACT_COD"] == "ATU")                     # Code activité correspondant à "forfait d'accueil et de traitement des urgences"
    &
    (~
     df_mco_fbstc.index.get_level_values(0)                # Suppression des séjours en double APHP, HCL, AP-HM
     .isin(ETA_NUM_doubles))
)
atu_pub = df_mco_fbstc[filtres_mco_fbstc].groupby('ETA_NUM')['ACT_NBR'].agg('sum')
atu_pub = atu_pub.reset_index(name='NB_URG_EXT')

# Passages aux urgences sans hospitalisation (sans séjour) dans le privé
atu_priv = df_mco_fb[
                     df_mco_fb["ACT_COD"] == "ATU"          # Code activité correspondant à "forfait d'accueil et de traitement des urgences"
                     ].groupby('ETA_NUM')['ACT_NBR'].agg('sum')
atu_priv = atu_priv.reset_index(name='NB_URG_EXT')

# Combiner tables atu_pub et atu_priv
atu = atu_pub.append(atu_priv)

# Jointure entre urgences sans hospit et avec hospit
#total_urg = atu.join(pass_urg_hospi, on='ETA_NUM', how='outer', lsuffix='_atu', rsuffix='hosp')
total_urg = pd.merge(atu, pass_urg_hospi, on='ETA_NUM', how='outer')

# Remplacer valeurs manquantes par 0
total_urg.fillna(0, inplace=True)

total_urg.head(5)  

,ETA_NUM,NB_URG_EXT,NB_URG_HOSP
0,146465501,512.0,0.0
1,146465501,508.0,0.0
2,161884379,923.0,1.0
3,161884379,805.0,1.0
4,319598408,421.0,0.0


## Exercice 4 : Identifier le nombre d'actes médicaux exécutés par établissement pour un acte spécifique - Exemple de l'exérèse de lésion du tronc cérébral par craniotomie
Dans cet exercice, nous allons comptabiliser le nombre d'actes exécutés lors de séjours hospitaliers pour un acte spécifique par établissement l'année aa. Un acte médical dans le SNDS est classifié suivant la Classification Commune des Actes Médicaux CCAM. L'exercice fait appel à la table des actes T_MCOaaA. Il faudra penser à supprimer les actes associés à des séjours en erreur (filtres des exercices précédents) qu'on repèrera dans la table des séjours T_MCOaaB. Il faut également supprimer les séjours en double APHP, HCL, AP-HM grâce à la variable ETA_NUM (variable présente dans T_MCOaaA).Commençons tout d'abord par chercher dans la nomenclature IR_CCAM_V54, le code CCAM de l'acte qu'on cherche à étudier. Vous pouvez trouver les différentes nomenclatures dans le dictionnaire SNDS au niveau de l'onglet "Recherche dans les nomenclatures" et effectuer une recherche par mots clés par exemple. Ici, focalisons-nous sur l'exérèse de lésion du tronc cérébral par craniotomie, qui correspond au code CCAM "AAFA003". Nous allons repérer les établissements ayant exécuté cet acte l'année aa et le nombre d'actes effectués.

In [22]:
filtres_mco_a = (
    (df_mco_a["CDC_ACT"]=='AAFA003')                 # Acte CCAM recherché
    #& (df_mco_a["PHA_ACT"]==0)                      # Evite de sélectionner plusieurs lignes si acte en plusieurs phases
    #& (df_mco_a["CV_ACT"]==1)                       # Activité du médecin : évite de sélectionner plusieurs lignes si plusieurs professionnels interviennent
    & 
    (df_mco_a["ETA_NUM_RSA_NUM"]
    .isin(df_mco_b.loc[filtres_mco_b,"ETA_NUM_RSA_NUM"])) # Suppression des séjours en erreur 
    &
    (~
     df_mco_a.index.get_level_values(0)                   # Suppression des séjours en double APHP, HCL, AP-HM
     .isin(ETA_NUM_doubles)) 
)

nb_AAFA003 = df_mco_a[filtres_mco_a].groupby('ETA_NUM')['NBR_EXE_ACT'].agg('sum')
nb_AAFA003 = nb_AAFA003.reset_index(name='NB_ACT')

nb_AAFA003.head(5)  

,ETA_NUM,NB_ACT
0,113270896,83
1,174974839,8
2,185093402,16
3,225176830,67
4,263149864,68
